Importing modules

In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler

import numpy as np
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import PoissonRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_poisson_deviance, classification_report, confusion_matrix
from scipy.optimize import minimize

from sklearn.model_selection import train_test_split

from sklearn.ensemble import HistGradientBoostingRegressor

import joblib



Load data

In [2]:
df = pd.read_csv('Matches_with_last5.csv')

df.tail()

C:\Users\lendv\AppData\Local\Temp\ipykernel_4880\682690520.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Matches_with_last5.csv')


,Division,MatchDate,MatchTime,HomeTeam,AwayTeam,HomeElo,AwayElo,Form3Home,Form5Home,Form3Away,...,Home_Last5_GoalsAgainst,Home_Last5_Wins,Home_Last5_Draws,Home_Last5_Losses,Away_Last5_Points,Away_Last5_GoalsFor,Away_Last5_GoalsAgainst,Away_Last5_Wins,Away_Last5_Draws,Away_Last5_Losses
228372,E0,2025-02-26,19:30:00,Tottenham,Man City,1785.53,1926.48,9.0,9.0,3.0,...,6.0,3.0,0.0,2.0,9.0,14.0,8.0,3.0,0.0,2.0
228373,E0,2025-02-26,19:30:00,Brentford,Everton,1738.05,1731.52,6.0,9.0,5.0,...,5.0,3.0,0.0,2.0,11.0,11.0,5.0,3.0,2.0,0.0
228374,E0,2025-02-26,19:30:00,Man United,Ipswich,1757.62,1584.51,1.0,4.0,1.0,...,8.0,1.0,1.0,3.0,1.0,4.0,17.0,0.0,1.0,4.0
228375,E0,2025-02-26,19:30:00,Nott'm Forest,Arsenal,1788.28,1999.49,3.0,6.0,6.0,...,13.0,2.0,0.0,3.0,10.0,10.0,4.0,3.0,1.0,1.0
228376,E0,2025-02-26,20:15:00,Liverpool,Newcastle,1995.58,1826.18,7.0,11.0,3.0,...,5.0,3.0,2.0,0.0,6.0,9.0,14.0,2.0,0.0,3.0


Check null values

In [3]:
df.isnull().sum()

Division                        0
MatchDate                       0
MatchTime                  131485
HomeTeam                        0
AwayTeam                        0
HomeElo                     88467
AwayElo                     88535
Form3Home                    1500
Form5Home                    1500
Form3Away                    1500
Form5Away                    1500
FTHome                          3
FTAway                          3
FTResult                        3
HTHome                      54580
HTAway                      54580
HTResult                    54580
HomeShots                  115822
AwayShots                  115819
HomeTarget                 116628
AwayTarget                 116625
HomeFouls                  116584
AwayFouls                  116584
HomeCorners                116194
AwayCorners                116194
HomeYellow                 111259
AwayYellow                 111258
HomeRed                    111258
AwayRed                    111260
OddHome       

Feature scaling

In [4]:
keep_cols = ['HomeElo', 'AwayElo', 'FTHome', 'FTAway', 'Home_Last5_Points',
             'Home_Last5_GoalsFor', 'Home_Last5_GoalsAgainst', 'Home_Last5_Wins',
             'Home_Last5_Draws', 'Home_Last5_Losses', 'Away_Last5_Points',
             'Away_Last5_GoalsFor', 'Away_Last5_GoalsAgainst', 'Away_Last5_Wins',
             'Away_Last5_Draws', 'Away_Last5_Losses']

df = df[keep_cols]

df.tail()

,HomeElo,AwayElo,FTHome,FTAway,Home_Last5_Points,Home_Last5_GoalsFor,Home_Last5_GoalsAgainst,Home_Last5_Wins,Home_Last5_Draws,Home_Last5_Losses,Away_Last5_Points,Away_Last5_GoalsFor,Away_Last5_GoalsAgainst,Away_Last5_Wins,Away_Last5_Draws,Away_Last5_Losses
228372,1785.53,1926.48,0.0,1.0,9.0,10.0,6.0,3.0,0.0,2.0,9.0,14.0,8.0,3.0,0.0,2.0
228373,1738.05,1731.52,1.0,1.0,9.0,7.0,5.0,3.0,0.0,2.0,11.0,11.0,5.0,3.0,2.0,0.0
228374,1757.62,1584.51,3.0,2.0,4.0,4.0,8.0,1.0,1.0,3.0,1.0,4.0,17.0,0.0,1.0,4.0
228375,1788.28,1999.49,0.0,0.0,6.0,14.0,13.0,2.0,0.0,3.0,10.0,10.0,4.0,3.0,1.0,1.0
228376,1995.58,1826.18,2.0,0.0,11.0,10.0,5.0,3.0,2.0,0.0,6.0,9.0,14.0,2.0,0.0,3.0


Drop na rows

In [5]:
df.dropna(inplace=True)

df.head()

,HomeElo,AwayElo,FTHome,FTAway,Home_Last5_Points,Home_Last5_GoalsFor,Home_Last5_GoalsAgainst,Home_Last5_Wins,Home_Last5_Draws,Home_Last5_Losses,Away_Last5_Points,Away_Last5_GoalsFor,Away_Last5_GoalsAgainst,Away_Last5_Wins,Away_Last5_Draws,Away_Last5_Losses
19,1635.08,1698.35,1.0,1.0,3.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0,2.0,0.0,1.0,0.0
20,1776.69,1651.93,2.0,5.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
21,1622.39,1691.36,3.0,0.0,1.0,2.0,2.0,0.0,1.0,0.0,3.0,3.0,1.0,1.0,0.0,0.0
22,1484.23,1450.63,1.0,1.0,0.0,1.0,2.0,0.0,0.0,1.0,1.0,3.0,3.0,0.0,1.0,0.0
23,1581.55,1543.76,1.0,1.0,0.0,1.0,3.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0


Check all value types

In [6]:
is_numeric = df.applymap(lambda x: pd.api.types.is_number(x))
all_numeric = is_numeric.all().all()
all_numeric

C:\Users\lendv\AppData\Local\Temp\ipykernel_4880\263059098.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  is_numeric = df.applymap(lambda x: pd.api.types.is_number(x))


np.True_

In [7]:
len(df)

130368

Separate X and Y

In [8]:
feat_data = df[['HomeElo', 'AwayElo', 'Home_Last5_Points',
             'Home_Last5_GoalsFor', 'Home_Last5_GoalsAgainst', 'Home_Last5_Wins',
             'Home_Last5_Draws', 'Home_Last5_Losses', 'Away_Last5_Points',
             'Away_Last5_GoalsFor', 'Away_Last5_GoalsAgainst', 'Away_Last5_Wins',
             'Away_Last5_Draws', 'Away_Last5_Losses']]

scaler = StandardScaler()
original_cols = feat_data.columns
feat_data = scaler.fit_transform(feat_data)
feat_data = pd.DataFrame(feat_data, columns=original_cols)

label_data = df[['FTHome', 'FTAway']].copy()

feat_data.head()


,HomeElo,AwayElo,Home_Last5_Points,Home_Last5_GoalsFor,Home_Last5_GoalsAgainst,Home_Last5_Wins,Home_Last5_Draws,Home_Last5_Losses,Away_Last5_Points,Away_Last5_GoalsFor,Away_Last5_GoalsAgainst,Away_Last5_Wins,Away_Last5_Draws,Away_Last5_Losses
0,0.616009,1.035775,-1.136067,-1.786428,-2.238944,-0.666517,-1.336527,-1.573571,-1.829708,-1.498028,-1.521772,-1.576023,-0.334653,-1.520498
1,1.555790,0.727775,-1.750609,-1.786428,-1.900319,-1.513538,-0.336577,-1.573571,-2.136844,-2.141859,-1.521772,-1.576023,-1.333851,-0.661022
2,0.531793,0.989396,-1.750609,-1.460037,-1.561693,-1.513538,-0.336577,-1.573571,-1.215436,-1.176112,-1.864794,-0.735088,-1.333851,-1.520498
3,-0.385093,-0.607862,-2.057880,-1.786428,-1.561693,-1.513538,-1.336527,-0.724974,-1.829708,-1.176112,-1.178750,-1.576023,-0.334653,-1.520498
4,0.260762,0.010061,-2.057880,-1.786428,-1.223068,-1.513538,-1.336527,-0.724974,-2.136844,-2.141859,-1.864794,-1.576023,-1.333851,-0.661022


In [10]:
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [9]:
label_data.head()

,FTHome,FTAway
19,1.0,1.0
20,2.0,5.0
21,3.0,0.0
22,1.0,1.0
23,1.0,1.0


XGB regressor

In [24]:
X_train, X_test, y_train, y_test = train_test_split(
    feat_data, label_data, test_size=0.4, random_state=42
)

In [46]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

# Extended parameter grid
param_grid = {
    'n_estimators': [100, 200, 400, 500],        # number of boosting rounds
    'learning_rate': [0.001, 0.01, 0.05, 0.1],    # smaller lr with higher n_estimators
    'max_depth': [2, 3, 5],                     # deeper trees
    'subsample': [0.4, 0.6, 0.8],                      # row sampling
    'min_child_weight': [1, 3, 5],                  # min sum hessian in a child
    'reg_lambda': [0.1, 0.1, 1],                     # L2 regularization
}

# Enable GPU if available
xgb = XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
    tree_method='hist',   # enables GPU acceleration
    device='cuda'
)

# GridSearch for Home
grid_search_home = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=2,
    n_jobs=4
)

grid_search_home.fit(X_train, y_train['FTHome'])
print("Best Home Params:", grid_search_home.best_params_)

# GridSearch for Away
grid_search_away = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=2,
    n_jobs=4
)

grid_search_away.fit(X_train, y_train['FTAway'])
print("Best Away Params:", grid_search_away.best_params_)

# Best models
best_home_model = grid_search_home.best_estimator_
best_away_model = grid_search_away.best_estimator_

# Predictions
lambda_home = best_home_model.predict(X_test)
lambda_away = best_away_model.predict(X_test)

print("Home MSE:", mean_squared_error(y_test['FTHome'], lambda_home))
print("Away MSE:", mean_squared_error(y_test['FTAway'], lambda_away))


Fitting 3 folds for each of 1296 candidates, totalling 3888 fits
Best Home Params: {'learning_rate': 0.05, 'max_depth': 2, 'min_child_weight': 5, 'n_estimators': 400, 'reg_lambda': 1, 'subsample': 0.4}
Fitting 3 folds for each of 1296 candidates, totalling 3888 fits
Best Away Params: {'learning_rate': 0.05, 'max_depth': 2, 'min_child_weight': 3, 'n_estimators': 400, 'reg_lambda': 1, 'subsample': 0.6}
Home MSE: 1.4329100063163607
Away MSE: 1.1314050824160475


In [47]:
print("Home MAE:", mean_absolute_error(y_test['FTHome'], lambda_home))
print("Away MAE:", mean_absolute_error(y_test['FTAway'], lambda_away))

Home MAE: 0.952581618054061
Away MAE: 0.8264318604593474


In [ ]:
models = {
    "home": best_home_model,
    "away": best_away_model
}

joblib.dump(models, "xgb_models.pkl")

['xgb_models.pkl']

Poisson regression

In [ ]:
from scipy.stats import poisson

max_goals = 10

def match_probabilities(l_home, l_away, max_goals=5):
    probs = np.zeros((max_goals+1, max_goals+1))
    for i in range(max_goals+1):
        for j in range(max_goals+1):
            probs[i,j] = poisson.pmf(i, l_home) * poisson.pmf(j, l_away)
    return probs

In [ ]:
results = []

for i in range(len(X_test)):
    probs = match_probabilities(lambda_home[i], lambda_away[i], max_goals)
    p_home_win = np.tril(probs, -1).sum()
    p_away_win = np.triu(probs, 1).sum()
    p_draw = np.trace(probs)
    most_likely_score = np.unravel_index(np.argmax(probs), probs.shape)

    results.append({
        'Expected_Home_Goals': lambda_home[i],
        'Expected_Away_Goals': lambda_away[i],
        'P_HomeWin': p_home_win,
        'P_Draw': p_draw,
        'P_AwayWin': p_away_win,
        'Most_Likely_Score': f"{most_likely_score[0]}-{most_likely_score[1]}"
    })

predictions = pd.DataFrame(results)

# Combine with actuals (optional)
final = predictions.copy()
final['Actual_FTHome'] = y_test['FTHome'].values
final['Actual_FTAway'] = y_test['FTAway'].values

print(final.head(10))

   Expected_Home_Goals  Expected_Away_Goals  P_HomeWin    P_Draw  P_AwayWin  \
0             1.160615             1.513700   0.291263  0.254910   0.453827   
1             1.631166             1.028086   0.514385  0.246804   0.238809   
2             1.860634             1.072053   0.558054  0.225778   0.216163   
3             1.740340             0.998890   0.547827  0.236307   0.215863   
4             0.659113             2.272809   0.091738  0.168503   0.739732   
5             1.398725             1.083212   0.441565  0.267071   0.291364   
6             1.564031             1.116668   0.476679  0.251752   0.271569   
7             1.440482             1.180043   0.430597  0.260071   0.309331   
8             1.500921             0.926543   0.506191  0.260874   0.232934   
9             1.354695             1.137998   0.417317  0.268569   0.314114   

  Most_Likely_Score  Actual_FTHome  Actual_FTAway  
0               1-1            3.0            1.0  
1               1-1       

In [ ]:
final

,Expected_Home_Goals,Expected_Away_Goals,P_HomeWin,P_Draw,P_AwayWin,Most_Likely_Score,Actual_FTHome,Actual_FTAway
0,1.160615,1.513700,0.291263,0.254910,0.453827,1-1,3.0,1.0
1,1.631166,1.028086,0.514385,0.246804,0.238809,1-1,3.0,3.0
2,1.860634,1.072053,0.558054,0.225778,0.216163,1-1,3.0,3.0
3,1.740340,0.998890,0.547827,0.236307,0.215863,1-0,2.0,1.0
4,0.659113,2.272809,0.091738,0.168503,0.739732,0-2,0.0,1.0
...,...,...,...,...,...,...,...,...
26069,1.347493,1.062938,0.432428,0.272430,0.295142,1-1,1.0,2.0
26070,2.054416,0.874414,0.647208,0.202273,0.150509,2-0,4.0,1.0
26071,1.140689,1.205785,0.343984,0.280153,0.375863,1-1,4.0,2.0
26072,1.495430,1.018163,0.482384,0.259947,0.257669,1-1,2.0,0.0


In [ ]:
count_values = final['Most_Likely_Score'].value_counts()
print(count_values)

Most_Likely_Score
1-1    15729
1-0     6277
2-0     2450
0-1      838
0-2      445
3-0      133
2-1      101
1-2       87
0-3        6
3-1        5
4-0        2
0-0        1
Name: count, dtype: int64


XGBoost

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    feat_data, label_data, test_size=0.4, random_state=42
)

In [11]:
X_home = X_train.copy()
y_home = y_train['FTHome'].copy()

X_away = X_train.copy()
y_away = y_train['FTAway'].copy()


In [12]:
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, KFold
import numpy as np

def optimize_xgb(X, y, n_trials=100):
    def objective(trial):
        params = {
            # Core parameters
            "n_estimators": trial.suggest_int("n_estimators", 10, 2000),
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 2.3, log=True),

            # Tree complexity
            "max_depth": trial.suggest_int("max_depth", 2, 12),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 15),
            "max_leaves": trial.suggest_int("max_leaves", 0, 512),  # 0 = unlimited
            "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"]),

            # Sampling
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
            "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.5, 1.0),

            # Regularization
            "gamma": trial.suggest_float("gamma", 0, 5),
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),

            # Other
            "scale_pos_weight": trial.suggest_float("scale_pos_weight", 0.5, 2.0),
            "objective": "reg:squarederror",
            "tree_method": "hist",  # or "gpu_hist" if GPU available
            "random_state": 42,
        }

        model = XGBRegressor(**params)
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        scores = cross_val_score(model, X, y, cv=kf,
                                 scoring="neg_mean_absolute_error", n_jobs=-1)
        return np.mean(scores)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials, n_jobs=-1)

    print("Best params:", study.best_trial.params)
    return study.best_trial.params

print("🔹 Tuning HOME model...")
best_params_home = optimize_xgb(X_home, y_home)
home_model = XGBRegressor(**best_params_home)
home_model.fit(X_home, y_home)

# Evaluate on training data (or use X_test, y_test if available)
home_pred = home_model.predict(X_test)
home_mae = mean_absolute_error(y_test['FTHome'], home_pred)
print(f"🏠 Home Model MAE: {home_mae:.4f}")

print("\n🔹 Tuning AWAY model...")
best_params_away = optimize_xgb(X_away, y_away)
away_model = XGBRegressor(**best_params_away)
away_model.fit(X_away, y_away)

away_pred = away_model.predict(X_test)
away_mae = mean_absolute_error(y_test['FTAway'], away_pred)
print(f"🚗 Away Model MAE: {away_mae:.4f}")


c:\Users\lendv\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-10-05 17:20:53,738] A new study created in memory with name: no-name-c61700c8-f850-4a06-bcab-de4131cf16bb


🔹 Tuning HOME model...


[I 2025-10-05 17:21:34,798] Trial 5 finished with value: -0.9625903653362406 and parameters: {'n_estimators': 1797, 'learning_rate': 0.03196255333196018, 'max_depth': 2, 'min_child_weight': 5, 'max_leaves': 329, 'grow_policy': 'lossguide', 'subsample': 0.8520705083710378, 'colsample_bytree': 0.9114037504820076, 'colsample_bylevel': 0.5122280775589138, 'gamma': 1.5361592699644067, 'reg_alpha': 2.7528473430341024e-06, 'reg_lambda': 9.578681555605298e-08, 'scale_pos_weight': 0.6901783262124627}. Best is trial 5 with value: -0.9625903653362406.
[I 2025-10-05 17:21:37,982] Trial 7 finished with value: -1.8892431133145386e+17 and parameters: {'n_estimators': 1374, 'learning_rate': 1.9152074434150061, 'max_depth': 12, 'min_child_weight': 7, 'max_leaves': 265, 'grow_policy': 'lossguide', 'subsample': 0.7040736721765035, 'colsample_bytree': 0.9276057933101519, 'colsample_bylevel': 0.978968484168933, 'gamma': 1.5958214219411482, 'reg_alpha': 0.0003747252364260921, 'reg_lambda': 1.322692609636784

Best params: {'n_estimators': 702, 'learning_rate': 0.09138272988358752, 'max_depth': 3, 'min_child_weight': 6, 'max_leaves': 439, 'grow_policy': 'depthwise', 'subsample': 0.8620974298526729, 'colsample_bytree': 0.7160691359646386, 'colsample_bylevel': 0.7656666560786305, 'gamma': 3.007516853837185, 'reg_alpha': 0.005149928787241159, 'reg_lambda': 5.760458941317286e-06, 'scale_pos_weight': 1.8619632420682637}


[I 2025-10-05 17:28:10,864] A new study created in memory with name: no-name-31bc1583-49ea-4636-8175-2f698f35efde


🏠 Home Model MAE: 0.9438

🔹 Tuning AWAY model...


[I 2025-10-05 17:28:16,183] Trial 2 finished with value: -0.8736187747988741 and parameters: {'n_estimators': 435, 'learning_rate': 0.47788263863736835, 'max_depth': 12, 'min_child_weight': 6, 'max_leaves': 292, 'grow_policy': 'depthwise', 'subsample': 0.9013675830493799, 'colsample_bytree': 0.8352663337657322, 'colsample_bylevel': 0.6581970024155275, 'gamma': 1.841083675563175, 'reg_alpha': 0.007709862041390837, 'reg_lambda': 0.001435406422831808, 'scale_pos_weight': 0.9652759700897707}. Best is trial 2 with value: -0.8736187747988741.
[I 2025-10-05 17:28:25,706] Trial 1 finished with value: -0.9025446187935549 and parameters: {'n_estimators': 1129, 'learning_rate': 0.5308767592948904, 'max_depth': 6, 'min_child_weight': 4, 'max_leaves': 199, 'grow_policy': 'lossguide', 'subsample': 0.6616299904690832, 'colsample_bytree': 0.952374199886493, 'colsample_bylevel': 0.912197299046769, 'gamma': 3.404985602243107, 'reg_alpha': 1.2020475540013313e-06, 'reg_lambda': 1.1802626544325239e-07, 'sc

Best params: {'n_estimators': 754, 'learning_rate': 0.010014776780087243, 'max_depth': 5, 'min_child_weight': 7, 'max_leaves': 478, 'grow_policy': 'depthwise', 'subsample': 0.9691170260740455, 'colsample_bytree': 0.8715976738614736, 'colsample_bylevel': 0.9031180874638357, 'gamma': 4.660669293481116, 'reg_alpha': 6.8079887351954226e-06, 'reg_lambda': 5.856780292141553, 'scale_pos_weight': 1.9827204920082067}
🚗 Away Model MAE: 0.8189


In [53]:
joblib.dump(home_model, "home_xgb.pkl")
joblib.dump(away_model, "away_xgb.pkl")

['away_xgb.pkl']

Random Forest

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# HOME model
print("🔹 Training HOME Random Forest model...")

home_model = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1
)

home_model.fit(X_home, y_home)

home_pred = home_model.predict(X_test)
home_mae = mean_absolute_error(y_test['FTHome'], home_pred)
print(f"🏠 Home Model MAE: {home_mae:.4f}")


# AWAY model
print("\n🔹 Training AWAY Random Forest model...")

away_model = RandomForestRegressor(
    n_estimators=300,
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1
)

away_model.fit(X_away, y_away)

away_pred = away_model.predict(X_test)
away_mae = mean_absolute_error(y_test['FTAway'], away_pred)
print(f"🚗 Away Model MAE: {away_mae:.4f}")


🔹 Training HOME Random Forest model...
🏠 Home Model MAE: 0.9677

🔹 Training AWAY Random Forest model...
🚗 Away Model MAE: 0.8495


In [14]:
joblib.dump(home_model, "home_random_forest.pkl")
joblib.dump(away_model, "away_random_forest.pkl")

['away_random_forest.pkl']

Gradient Boosting

In [12]:
import optuna
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error
import numpy as np

def optimize_gb(X, y, n_trials=100):
    def objective(trial):
        params = {
            # Core boosting parameters
            "n_estimators": trial.suggest_int("n_estimators", 50, 2000),
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
            "subsample": trial.suggest_float("subsample", 0.5, 1.0),

            # Tree structure
            "max_depth": trial.suggest_int("max_depth", 2, 10),
            "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
            "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2", None]),

            # Loss function
            "loss": trial.suggest_categorical("loss", ["squared_error", "absolute_error", "huber"]),
            
            # Randomness
            "random_state": 42,
        }

        model = GradientBoostingRegressor(**params)
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        scores = cross_val_score(model, X, y, cv=kf,
                                 scoring="neg_mean_absolute_error", n_jobs=-1)
        return np.mean(scores)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials, n_jobs=-1)

    print("Best params:", study.best_trial.params)
    return study.best_trial.params


# Example usage:
print("🔹 Tuning HOME model...")
best_params_home = optimize_gb(X_home, y_home)
home_model = GradientBoostingRegressor(**best_params_home)
home_model.fit(X_home, y_home)

# Evaluate on training data (or use X_test, y_test if available)
home_pred = home_model.predict(X_test)
home_mae = mean_absolute_error(y_test['FTHome'], home_pred)
print(f"🏠 Home Model MAE: {home_mae:.4f}")

print("\n🔹 Tuning AWAY model...")
best_params_away = optimize_gb(X_away, y_away)
away_model = GradientBoostingRegressor(**best_params_away)
away_model.fit(X_away, y_away)

away_pred = away_model.predict(X_test)
away_mae = mean_absolute_error(y_test['FTAway'], away_pred)
print(f"🚗 Away Model MAE: {away_mae:.4f}")


c:\Users\lendv\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-10-06 08:12:37,478] A new study created in memory with name: no-name-dbcf9d06-39cb-4110-9605-004083fd49dd


🔹 Tuning HOME model...


[I 2025-10-06 08:22:34,290] Trial 12 finished with value: -0.9273833979822499 and parameters: {'n_estimators': 1373, 'learning_rate': 0.040687737740769885, 'subsample': 0.5893564677324034, 'max_depth': 3, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max_features': None, 'loss': 'absolute_error'}. Best is trial 12 with value: -0.9273833979822499.
[I 2025-10-06 08:23:32,960] Trial 15 finished with value: -0.950077249459528 and parameters: {'n_estimators': 913, 'learning_rate': 0.09939317429345021, 'subsample': 0.621415601902306, 'max_depth': 3, 'min_samples_split': 10, 'min_samples_leaf': 9, 'max_features': None, 'loss': 'huber'}. Best is trial 12 with value: -0.9273833979822499.
[I 2025-10-06 08:24:55,962] Trial 8 finished with value: -0.9698816878853055 and parameters: {'n_estimators': 906, 'learning_rate': 0.030312508994113998, 'subsample': 0.7222073439710099, 'max_depth': 9, 'min_samples_split': 12, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'loss': 'squared_error'}. Best is t

Best params: {'n_estimators': 1618, 'learning_rate': 0.0027366945950938514, 'subsample': 0.9690127706833245, 'max_depth': 7, 'min_samples_split': 8, 'min_samples_leaf': 1, 'max_features': None, 'loss': 'absolute_error'}


[I 2025-10-06 10:14:48,770] A new study created in memory with name: no-name-0f57f239-3b75-45fe-b9b7-9235c15bdef1


🏠 Home Model MAE: 0.9133

🔹 Tuning AWAY model...


[I 2025-10-06 10:17:27,249] Trial 0 finished with value: -0.806649281225172 and parameters: {'n_estimators': 231, 'learning_rate': 0.0024701944533922983, 'subsample': 0.540086020140655, 'max_depth': 10, 'min_samples_split': 18, 'min_samples_leaf': 7, 'max_features': 'log2', 'loss': 'absolute_error'}. Best is trial 0 with value: -0.806649281225172.
[I 2025-10-06 10:23:58,625] Trial 6 finished with value: -0.8304912859141359 and parameters: {'n_estimators': 802, 'learning_rate': 0.01845739328147622, 'subsample': 0.6733812842297744, 'max_depth': 3, 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'loss': 'squared_error'}. Best is trial 0 with value: -0.806649281225172.
[I 2025-10-06 10:32:20,005] Trial 14 finished with value: -0.8385279214819725 and parameters: {'n_estimators': 687, 'learning_rate': 0.07394763462604725, 'subsample': 0.686960196182256, 'max_depth': 5, 'min_samples_split': 18, 'min_samples_leaf': 6, 'max_features': 'sqrt', 'loss': 'squared_error'}. Be

Best params: {'n_estimators': 1879, 'learning_rate': 0.003200785380297327, 'subsample': 0.8784959807859897, 'max_depth': 6, 'min_samples_split': 12, 'min_samples_leaf': 1, 'max_features': None, 'loss': 'absolute_error'}
🚗 Away Model MAE: 0.7975


In [13]:
joblib.dump(home_model, "home_gradient_boosting.pkl")
joblib.dump(away_model, "away_gradient_boosting.pkl")

['away_gradient_boosting.pkl']

SVR

In [12]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error

# HOME model
print("🔹 Training HOME SVR model...")

home_model = make_pipeline(
    SVR(
        C=1.0,           # Regularization parameter
        epsilon=0.1,     # Insensitive loss
        kernel='rbf',    # Radial Basis Function kernel
        gamma='scale'    # Kernel coefficient
    )
)

home_model.fit(X_home, y_home)

home_pred = home_model.predict(X_test)
home_mae = mean_absolute_error(y_test['FTHome'], home_pred)
print(f"🏠 Home Model MAE: {home_mae:.4f}")


# AWAY model
print("\n🔹 Training AWAY SVR model...")

away_model = make_pipeline(
    SVR(
        C=1.0,
        epsilon=0.1,
        kernel='rbf',
        gamma='scale'
    )
)

away_model.fit(X_away, y_away)

away_pred = away_model.predict(X_test)
away_mae = mean_absolute_error(y_test['FTAway'], away_pred)
print(f"🚗 Away Model MAE: {away_mae:.4f}")


🔹 Training HOME SVR model...
🏠 Home Model MAE: 0.9302

🔹 Training AWAY SVR model...
🚗 Away Model MAE: 0.8143


In [13]:
joblib.dump(home_model, "home_svr.pkl")
joblib.dump(away_model, "away_svr.pkl")

['away_svr.pkl']

HistGradientBoosting

In [13]:
import optuna
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error
import numpy as np

def optimize_hgb(X, y, n_trials=100):
    def objective(trial):
        params = {
            # Core boosting params
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.3, log=True),
            "max_iter": trial.suggest_int("max_iter", 50, 2000),
            "max_leaf_nodes": trial.suggest_int("max_leaf_nodes", 15, 255),
            "max_depth": trial.suggest_int("max_depth", 2, 16),
            "min_samples_leaf": trial.suggest_int("min_samples_leaf", 5, 100),
            "l2_regularization": trial.suggest_float("l2_regularization", 1e-8, 10.0, log=True),
            "max_bins": trial.suggest_int("max_bins", 64, 255),
            "early_stopping": False,  # disable during CV
            "scoring": "loss",
            "random_state": 42,
        }


        model = HistGradientBoostingRegressor(**params)
        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        scores = cross_val_score(model, X, y, cv=kf,
                                 scoring="neg_mean_absolute_error", n_jobs=-1)
        return np.mean(scores)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials, n_jobs=-1)

    print("Best params:", study.best_trial.params)
    return study.best_trial.params


# Example usage:
print("🔹 Tuning HOME model...")
best_params_home = optimize_hgb(X_home, y_home)
home_model = HistGradientBoostingRegressor(**best_params_home)
home_model.fit(X_home, y_home)

home_pred = home_model.predict(X_test)
home_mae = mean_absolute_error(y_test['FTHome'], home_pred)
print(f"🏠 Home Model MAE: {home_mae:.4f}")

print("\n🔹 Tuning AWAY model...")
best_params_away = optimize_hgb(X_away, y_away)
away_model = HistGradientBoostingRegressor(**best_params_away)
away_model.fit(X_away, y_away)

away_pred = away_model.predict(X_test)
away_mae = mean_absolute_error(y_test['FTAway'], away_pred)
print(f"🚗 Away Model MAE: {away_mae:.4f}")


[I 2025-10-11 16:15:48,374] A new study created in memory with name: no-name-417d4dfd-b38d-400c-81dc-78ca49899e76


🔹 Tuning HOME model...


[I 2025-10-11 16:15:54,280] Trial 1 finished with value: -1.0034917752147925 and parameters: {'learning_rate': 0.0049320398190244455, 'max_iter': 188, 'max_leaf_nodes': 137, 'max_depth': 2, 'min_samples_leaf': 25, 'l2_regularization': 1.571997408856443e-08, 'max_bins': 255}. Best is trial 1 with value: -1.0034917752147925.
[I 2025-10-11 16:15:56,061] Trial 0 finished with value: -0.9676558029014979 and parameters: {'learning_rate': 0.009688200904655433, 'max_iter': 140, 'max_leaf_nodes': 76, 'max_depth': 16, 'min_samples_leaf': 58, 'l2_regularization': 0.5573527345926199, 'max_bins': 88}. Best is trial 0 with value: -0.9676558029014979.
[I 2025-10-11 16:16:44,447] Trial 4 finished with value: -0.9585931300178304 and parameters: {'learning_rate': 0.0390621866668769, 'max_iter': 1414, 'max_leaf_nodes': 137, 'max_depth': 4, 'min_samples_leaf': 31, 'l2_regularization': 0.000949653243448047, 'max_bins': 94}. Best is trial 4 with value: -0.9585931300178304.
[I 2025-10-11 16:17:08,120] Trial 

Best params: {'learning_rate': 0.010946521717759575, 'max_iter': 1992, 'max_leaf_nodes': 168, 'max_depth': 3, 'min_samples_leaf': 100, 'l2_regularization': 0.1433429087644945, 'max_bins': 175}


[I 2025-10-11 16:29:15,595] A new study created in memory with name: no-name-845589bb-29a8-4365-ac6c-475f1864a6d7


🏠 Home Model MAE: 0.9535

🔹 Tuning AWAY model...


[I 2025-10-11 16:29:49,750] Trial 0 finished with value: -0.8735168039622421 and parameters: {'learning_rate': 0.09512611950767944, 'max_iter': 1138, 'max_leaf_nodes': 164, 'max_depth': 7, 'min_samples_leaf': 18, 'l2_regularization': 0.00015579653807103097, 'max_bins': 175}. Best is trial 0 with value: -0.8735168039622421.
[I 2025-10-11 16:30:10,330] Trial 2 finished with value: -0.8423430509889667 and parameters: {'learning_rate': 0.027580469860749718, 'max_iter': 419, 'max_leaf_nodes': 248, 'max_depth': 9, 'min_samples_leaf': 9, 'l2_regularization': 0.004426940149794415, 'max_bins': 190}. Best is trial 2 with value: -0.8423430509889667.
[I 2025-10-11 16:30:35,347] Trial 4 finished with value: -0.9305291843604795 and parameters: {'learning_rate': 0.25565875110178976, 'max_iter': 863, 'max_leaf_nodes': 110, 'max_depth': 12, 'min_samples_leaf': 13, 'l2_regularization': 1.859400640579355e-05, 'max_bins': 173}. Best is trial 2 with value: -0.8423430509889667.
[I 2025-10-11 16:30:37,550] T

Best params: {'learning_rate': 0.0018622855517445678, 'max_iter': 891, 'max_leaf_nodes': 51, 'max_depth': 13, 'min_samples_leaf': 37, 'l2_regularization': 0.0008362301937444748, 'max_bins': 235}
🚗 Away Model MAE: 0.8254


In [14]:
joblib.dump(home_model, "home_histgrad.pkl")
joblib.dump(away_model, "away_histgrad.pkl")

['away_histgrad.pkl']

MLP

In [ ]:
import optuna
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_error
import numpy as np

def optimize_mlp(X, y, n_trials=20):
    def objective(trial):
        params = {
            # Network architecture
            "hidden_layer_sizes": tuple(
                trial.suggest_categorical("hidden_layer_sizes", [
                    (50,), (100,), (50, 50), (100, 50), (100, 100), (200, 100)
                ])
            ),
            "activation": trial.suggest_categorical("activation", ["relu", "tanh", "logistic"]),
            "solver": trial.suggest_categorical("solver", ["adam", "lbfgs", "sgd"]),
            "alpha": trial.suggest_float("alpha", 1e-6, 1e-1, log=True),  # L2 regularization
            "learning_rate": trial.suggest_categorical("learning_rate", ["constant", "adaptive"]),
            "learning_rate_init": trial.suggest_float("learning_rate_init", 1e-5, 1e-1, log=True),
            "batch_size": trial.suggest_categorical("batch_size", ["auto", 16, 32, 64, 128]),
            "max_iter":  trial.suggest_int("max_iter", 200, 2000),
            "shuffle": True,
            "random_state": 42,
        }

        # Prevent slow SGD runs with bad configs
        if params["solver"] == "sgd":
            params["momentum"] = trial.suggest_float("momentum", 0.5, 0.99)
            params["nesterovs_momentum"] = trial.suggest_categorical("nesterovs_momentum", [True, False])
        else:
            params["momentum"] = 0.9  # default for non-SGD solvers

        model = MLPRegressor(**params)

        kf = KFold(n_splits=5, shuffle=True, random_state=42)
        scores = cross_val_score(model, X, y, cv=kf,
                                 scoring="neg_mean_absolute_error", n_jobs=-1)
        return np.mean(scores)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials, n_jobs=-1)

    print("Best params:", study.best_trial.params)
    return study.best_trial.params


# Example usage:
print("🔹 Tuning HOME model...")
best_params_home = optimize_mlp(X_home, y_home)
home_model = MLPRegressor(**best_params_home)
home_model.fit(X_home, y_home)

home_pred = home_model.predict(X_test)
home_mae = mean_absolute_error(y_test['FTHome'], home_pred)
print(f"🏠 Home Model MAE: {home_mae:.4f}")

print("\n🔹 Tuning AWAY model...")
best_params_away = optimize_mlp(X_away, y_away)
away_model = MLPRegressor(**best_params_away)
away_model.fit(X_away, y_away)

away_pred = away_model.predict(X_test)
away_mae = mean_absolute_error(y_test['FTAway'], away_pred)
print(f"🚗 Away Model MAE: {away_mae:.4f}")


[I 2025-10-13 14:03:33,900] A new study created in memory with name: no-name-b632257b-d725-4aa9-9d27-310289bd8b7d


🔹 Tuning HOME model...


c:\Users\lendv\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:518: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (50,) which is of type tuple.
  warnings.warn(message)
c:\Users\lendv\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:518: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
c:\Users\lendv\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:518: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (50, 50) which is of type tuple.
  warnings.warn(message)
c:\Users\lendv\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\distributions.py:518: Use

In [13]:
joblib.dump(home_model, "home_mlp.pkl")
joblib.dump(away_model, "away_mlp.pkl")

['away_mlp.pkl']